In [20]:
pip install -q -U keras-tuner

In [21]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [22]:
fashion_mnist = keras.datasets.fashion_mnist

In [23]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [24]:
train_images = train_images/255.0
test_images = test_images/255.0

In [25]:
train_images[0].shape

(28, 28)

In [26]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [31]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters = hp.Int('conv_1_filter', min_value = 32, max_value = 128, step = 16),
        kernel_size = hp.Choice('conv_1_kernel', values = [3,5]),
        activation = 'relu',
        input_shape = (28,28,1)
    ),
    keras.layers.Conv2D(
        filters = hp.Int('conv_2_filter', min_value = 32, max_value = 64, step = 16),
        kernel_size = hp.Choice('conv_2_kernel', values = [3,5]),
        activation = 'relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step = 16 ),
        activation = 'relu'
    ),
    keras.layers.Dense(10, activation='softmax')#output layer
  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2, 1e-3])),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
  return model

In [32]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [35]:
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials=5,
                            directory='output',
                            project_name="Mnist_Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist_Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist_Fashion/tuner0.json


In [34]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 1 Complete [00h 37m 17s]
val_accuracy: 0.8694999814033508

Best val_accuracy So Far: 0.8694999814033508
Total elapsed time: 00h 37m 17s
INFO:tensorflow:Oracle triggered exit


In [36]:
model = tuner_search.get_best_models(num_models=1)[0]

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        179264    
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                991264    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 1,171,978
Trainable params: 1,171,978
Non-trainable params: 0
_________________________________________________________________
